In [1]:
! pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.9 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=2121d8f95d6b5b8bd95abb930cae125b0b50b767aa6863936d61761cbfcfbd3d
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [2]:
from IPython import get_ipython
from IPython.display import display
import requests
import feedparser
import os
import time
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import csv
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Updated Atom feed URL
feed_url = "https://caselaw.nationalarchives.gov.uk/atom.xml?"

In [4]:
# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [5]:
def scrape_case_content(topic_url):
    try:
        response = requests.get(topic_url, headers=headers)
        topic_soup = BeautifulSoup(response.text, 'html.parser')

        # Extract relevant content (adjust as needed)
        # Example: Extract all paragraph text
        sections = topic_soup.find_all('p')

        # Create XML elements
        case_element = ET.Element("case")
        for section in sections:
            text_element = ET.SubElement(case_element, "text")
            text_element.text = section.get_text()

        # Append to existing XML file or create a new one
        try:
            tree = ET.parse('/content/drive/MyDrive/all_case_data.xml')
            root = tree.getroot()
            root.append(case_element)
        except FileNotFoundError:
            root = ET.Element("cases")
            root.append(case_element)
            tree = ET.ElementTree(root)

        tree.write('/content/drive/MyDrive/all_case_data.xml', encoding='utf-8', xml_declaration=True)

        print(f"Scraped content from {topic_url}")

    except Exception as e:
        print(f"Error scraping {topic_url}: {e}")

In [6]:
# Fetch and parse the Atom feed
response = requests.get(feed_url, headers=headers)
feed = feedparser.parse(response.content)

# Extract judgment links
judgment_links = [entry.link for entry in feed.entries]

# Process each judgment link
for link in judgment_links:
    scrape_case_content(link)  # Directly use the link from the feed
    time.sleep(5)  # Add a delay to avoid overloading the server

Scraped content from https://caselaw.nationalarchives.gov.uk/ewhc/kb/2025/138
Scraped content from https://caselaw.nationalarchives.gov.uk/ewhc/ch/2025/135
Scraped content from https://caselaw.nationalarchives.gov.uk/ewhc/admin/2025/123
Scraped content from https://caselaw.nationalarchives.gov.uk/ewhc/comm/2025/140
Scraped content from https://caselaw.nationalarchives.gov.uk/ewca/crim/2025/52
Scraped content from https://caselaw.nationalarchives.gov.uk/ewhc/ch/2025/136
Scraped content from https://caselaw.nationalarchives.gov.uk/ewca/crim/2025/51
Scraped content from https://caselaw.nationalarchives.gov.uk/ewhc/admin/2025/137
Scraped content from https://caselaw.nationalarchives.gov.uk/ukftt/tc/2025/66
Scraped content from https://caselaw.nationalarchives.gov.uk/ukftt/grc/2025/73
Scraped content from https://caselaw.nationalarchives.gov.uk/ewhc/admin/2025/113
Scraped content from https://caselaw.nationalarchives.gov.uk/ewhc/tcc/2025/100
Scraped content from https://caselaw.nationalarch

In [7]:
# Count and display the number of case laws
try:
    tree = ET.parse('/content/drive/MyDrive/all_case_data.xml')
    root = tree.getroot()
    case_count = len(root.findall('case'))
    print(f"Total number of case laws downloaded: {case_count}")
except FileNotFoundError:
    print("XML file not found.")

Total number of case laws downloaded: 100
